In [ ]:
import glob as glob

import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def read_vars(fName, vName):
    ds = xr.open_dataset(fName)
    var = ds[vName]
    return var

In [ ]:
varUnits={
    'Temp':'degC',
    'Salt':'PSU'}

varCbar={
    'Temp':[-2, 28, 'Spectral_r'],
    'Salt':[33, 38, 'nipy_spectral']}

## Set inputs:
- Names of experiments.
- Path to data.
- Dates (yyyymmdd) and hours: `yyyymmddThh`

In [ ]:
data_rootPath = "/scratch3/NCEPDEV/climate/Jiande.Wang/working/scratch/C384-v17/OCEAN/"
expNames = ["B1", "B2", "B3"]

# set date 
#dateStr = "20250101" # start date
#hrStr = "06" # starting hour (on start date)
dateStr = "20250126" # end date
hrStr = "00" # ending hour (on end date)

# set (3-d) variable names and level
vNames = ["Temp", "Salt"]
zMax = 500 # maximum depth to plot
lon_plot = -10

PDY = dateStr + "T" + hrStr

# path where figures will be saved
out_dir = "/home/Santha.Akella/for_folks/JiandeW/Sep2025/figs/"

## Load datasets from all:
- WOA 2023.
- The experiments set in `expNames`.
- Dates.
- Plot user requested variables.
  1. Temperature.
  2. Salinity.

In [ ]:
# WOA 2023
WOA_ver="WOA 2023"
res, fPref, fSuff = ["025", "woa23_decav91C0_", "04"]
woa_rootPath = "/scratch3/NCEPDEV/climate/Jiande.Wang/OBS-data/WOA2023/" + res+"/"

monStr = "02" # which month? 01- 12?
# set (3-d) variable names
varStr = ["t", "s"]
woa_vNames = ["t_an", "s_an"]

fName_temp = sorted(glob.glob(woa_rootPath + fPref + varStr[0] + monStr + "_" + fSuff +".nc"))[0]
print(f"Read {WOA_ver} temperature from\t{fName_temp}")
woa_t = xr.open_dataset(fName_temp,decode_times=False)[woa_vNames[0]]

fName_salt = sorted(glob.glob(woa_rootPath + fPref + varStr[1] + monStr + "_" + fSuff +".nc"))[0]
print(f"Read {WOA_ver} salinity from\t{fName_salt}")
woa_s = xr.open_dataset(fName_salt, decode_times=False)[woa_vNames[1]]

## Plot `potential temperature`

In [ ]:
varName = vNames[0]

fig=plt.figure(figsize=(12, 6), clear=True, dpi=120)

for iexp, expName in enumerate(expNames):
    print(f"Processing data from:\t{expName}")
    
    dPath = data_rootPath+f"{expName}/gdas.{dateStr}/{hrStr}/model/ocean/history/"
    fName = sorted(glob.glob(dPath + f"gdas.ocean.t*z.inst.f000.nc"))[0]
    print(f"Reading:\t{fName}")
    var = read_vars(fName, varName)

    ax = fig.add_subplot(2, 2, iexp+1)
    var_section=var.sel(xh=lon_plot, method='nearest').sel(yh=slice(-70., 60)).sel(z_l=slice(0, zMax))
    im=var_section.plot(ax=ax, yincrease=False, vmin=varCbar[varName][0], vmax=varCbar[varName][1], cmap=varCbar[varName][2])
    ax.set_title(f'{expName} {PDY} [lon={lon_plot}]')
        
    if iexp<2:
        ax.set_xlabel("")
    if iexp==1 or iexp==3:
        ax.set_ylabel("")

ax = fig.add_subplot(2, 2, 4)
woa_sec=woa_t.sel(lon=lon_plot, method='nearest').sel(lat=slice(-70., 60)).sel(depth=slice(0, zMax))
woa_sec.plot(ax=ax, yincrease=False, vmin=varCbar[varName][0], vmax=varCbar[varName][1], cmap=varCbar[varName][2])
ax.set_title(f'{WOA_ver} MON[{monStr}] [lon={lon_plot}]')
ax.set_ylabel("")

## Plot `salinity`

In [ ]:
varName = vNames[1]

fig=plt.figure(figsize=(12, 6), clear=True, dpi=120)

for iexp, expName in enumerate(expNames):
    print(f"Processing data from:\t{expName}")
    
    dPath = data_rootPath+f"{expName}/gdas.{dateStr}/{hrStr}/model/ocean/history/"
    fName = sorted(glob.glob(dPath + f"gdas.ocean.t*z.inst.f000.nc"))[0]
    print(f"Reading:\t{fName}")
    var = read_vars(fName, varName)

    ax = fig.add_subplot(2, 2, iexp+1)
    var_section=var.sel(xh=lon_plot, method='nearest').sel(yh=slice(-70., 60)).sel(z_l=slice(0, zMax))
    im=var_section.plot(ax=ax, yincrease=False, vmin=varCbar[varName][0], vmax=varCbar[varName][1], cmap=varCbar[varName][2])
    ax.set_title(f'{expName} {PDY} [lon={lon_plot}]')
        
    if iexp<2:
        ax.set_xlabel("")
    if iexp==1 or iexp==3:
        ax.set_ylabel("")

ax = fig.add_subplot(2, 2, 4)
woa_sec=woa_s.sel(lon=lon_plot, method='nearest').sel(lat=slice(-70., 60)).sel(depth=slice(0, zMax))
woa_sec.plot(ax=ax, yincrease=False, vmin=varCbar[varName][0], vmax=varCbar[varName][1], cmap=varCbar[varName][2])
ax.set_title(f'{WOA_ver} {monStr} [lon={lon_plot}]')
ax.set_ylabel("")